In [1]:
import time
import urllib
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd

# Import python file froms different folder
# reference: https://stackoverflow.com/questions/4383571/importing-files-from-different-folders
import sys
path_to_files = '/Volumes/ExtremeSSD/github_repos/trading_options/'
sys.path.insert(1, path_to_files) # insert at 1
from config import accounts

# How to approach TDA authentication

https://developer.tdameritrade.com/content/phase-1-authentication-update-xml-based-api

Basically, TDA follows the following authentication steps for use of their APIs:


1. Create a new app in Developer account, which shall provide the following items for authentication: 

    - `client_id`, which is shared among all linked TdAmeritrade accounts.
    - `callback_url`, for applications running on local machines, it's a link to your localhost URL, which will be used to receive the **auth code** after successful authentication


2. For the first time authorization code, we need to go through authentication process, which is to use browser to visit a special page to complete user login and 2-way factor authentication (receiving sms code) to arrive at a url that contains the **auth code**. This process can be done automatically using `splinter Browser` and `selenium webdriver` libraries.


3. With the **auth code**, we can access the [authentication api](https://developer.tdameritrade.com/authentication/apis) to retrieve the first time **access token**, don't forget to retrieve **refresh token** as well, it's because:

    - **Access token** expires in 30 minutes, however
    - **refresh token** is valid for 90 days. 

    Once the old **access token** expires every 30 minutes, we can just use **refresh token** to retrieve a new valid **access token** without going through the authentication that involves receiving sms code again
    
    
4. Now, we can make authenticated requests to all sorts of APIs with a valid **access token**.

# Get Authorization Code

## Define the URL for Authentication

In [2]:
# credentials
account_number = accounts['margin']['accountNumber']
client_id = accounts['margin']['clientId']
callback_url = accounts['margin']['callbackUrl']
username = accounts['margin']['username']

import getpass
password = getpass.getpass(f'Enter password for user "{username}":')

# Define all components of the URL
url = f'https://auth.tdameritrade.com/auth?'
method = 'GET'
client_code = client_id + '@AMER.OAUTHAP'
payload = {'response_type': 'code', 
           'redirect_uri': callback_url, 
           'client_id': client_code}

# build the url
built_url = requests.Request(method, url, params=payload).prepare()
built_url = built_url.url

Enter password for user "catelinnx2":········


## Automatic Authentication using Browser

In [3]:
"""Invoke browser"""
# path to chrome driver
executable_path = {'executable_path': '/Users/catelinn/drivers_for_dev/chromedriver'}

# Set default behaviors as Chrome Browser
options = webdriver.ChromeOptions()

# make sure the window is maximized
options.add_argument('--start-maximized')

# make sure notification are off
options.add_argument('--disable-notifications')

# create a new browser object, which
# 1. by default is Firefox, here, I use 'chrome';
# 2. connect to my chrome webdriver path
# 3. set `headless` to `False` to see the activities in browser, `True` not to see
# 4. pass in `options` to customize browser
browser = Browser('chrome', **executable_path, headless = False, options = options)

"""Now, we can visit the user authentication page"""
browser.visit(built_url)

# === Inspect the page and find the following ===
# ===============================================
# userID box
username_box = browser.find_by_id('username0')

# password box
password_box = browser.find_by_id('password1')

# === Automatic Authentication   ====
# ===================================
# fill in username and password at login page
username_box.fill(username)
password_box.fill(password)
browser.find_by_id('accept').first.click()

# continue to receive sms code
time.sleep(1) #give it a second to load
browser.find_by_id('accept').click()

# enter sms code
time.sleep(1)
smscode = input('Enter sms code: ')
browser.find_by_id('smscode0').type(smscode)
browser.find_by_id('accept').first.click()

# confirm trust device
time.sleep(1)
browser.find_by_xpath('/html/body/form/main/fieldset/div/div[1]').click()
browser.find_by_id('accept').first.click()

# confirm authorization
time.sleep(1)
browser.find_by_id('accept').first.click()

# the final url that shall contains the access token
new_url = browser.url

# close the browser
browser.quit()

Enter sms code: 532985


## Parse the Auth Code

In [4]:
# grab the url and parse it to retrieve the authorization code
# this code last only 30 minutes, use it quickly to retrieve the "access token"
auth_code = urllib.parse.unquote(new_url.split('code=')[1])

# Get the Access Token

In [5]:
# Now we can use the uAuth API to retrieve the bearer token, which
# can be used to access other APIs
#https://developer.tdameritrade.com/authentication/apis
def get_first_access_token(auth_code, client_id, callback_url):
    
    # define the endpoint
    url = 'https://api.tdameritrade.com/v1/oauth2/token'

    # define the headers
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    # define the payload
    payload = {'grant_type': 'authorization_code',
              'access_type': 'offline',
              'code': auth_code,
              'client_id': client_id,
              'redirect_uri': callback_url}

    # post the data to get the token
    return requests.post(url, headers=headers, data=payload)

In [6]:
r = get_first_access_token(auth_code, client_id, callback_url)

In [7]:
r.json().keys()

dict_keys(['access_token', 'refresh_token', 'scope', 'expires_in', 'refresh_token_expires_in', 'token_type'])

In [8]:
access_token = r.json()['access_token']
refresh_token = r.json()['refresh_token']

In [9]:
# save the refresh token to file
import datetime
from datetime import date
expiry = (date.today() + datetime.timedelta(days=90*2)).strftime("%m%d%Y")
f = open('refresh_token.py', 'w')
f.write('refresh_token='+repr(refresh_token))
f.write('\nexpiry='+repr(expiry))
f.close()

# Refresh Access Token after It Expires

In [10]:
def refresh_access_token(refresh_token, client_id):
    # define the endpoint
    url = 'https://api.tdameritrade.com/v1/oauth2/token'

    # define the headers
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    # define the payload
    payload = {'grant_type': 'refresh_token',
               'client_id': client_id,
               'refresh_token': refresh_token}

    # post the data to get the token
    r = requests.post(url, headers=headers, data=payload)
    return r.json()['access_token']

In [11]:
access_token = refresh_access_token(refresh_token, client_id)

# Make Authenticated API Requests

Now we can make authenticated request to the [TDA apis](https://developer.tdameritrade.com/apis) with the access token. 


## Access Account Information


Let's try with the `accounts` api: ([API docs](https://developer.tdameritrade.com/account-access/apis/get/accounts-0))

In [12]:
# define the endpoint
endpoint = 'https://api.tdameritrade.com/v1/accounts'

# prepare the headers (per the docs)
headers = {'Authorization': f'Bearer {access_token}'}

# make a request
r = requests.get(url=endpoint, headers=headers)

# parse the content
data = r.json()

# data include list of accounts
for item in data:
    print(item.keys())

dict_keys(['securitiesAccount'])


In [13]:
# see how to access each account's details
data[0]['securitiesAccount'].keys()

dict_keys(['type', 'accountId', 'roundTrips', 'isDayTrader', 'isClosingOnlyRestricted', 'initialBalances', 'currentBalances', 'projectedBalances'])

In [14]:
# access the first account - type
data[0]['securitiesAccount']['type']

'MARGIN'

## View Transactions

Here's the [documentation for transaction APIs](https://developer.tdameritrade.com/transaction-history/apis)

In [15]:
# define the endpoint
endpoint = f'https://api.tdameritrade.com/v1/accounts/{account_number}/transactions'

# prepare the headers (per the docs)
headers = {'Authorization': f'Bearer {access_token}'}

# prams
params = {'type': 'ALL', 'symbol': 'SPY'} 

# make a request
r = requests.get(url=endpoint, headers=headers, params=params)

# parse the content and convert to pandas df
data = pd.DataFrame(r.json())

In [16]:
# check avaialble fields
data.columns

Index(['type', 'subAccount', 'settlementDate', 'orderId', 'netAmount',
       'transactionDate', 'orderDate', 'transactionSubType', 'transactionId',
       'cashBalanceEffectFlag', 'description', 'fees', 'transactionItem'],
      dtype='object')

In [17]:
# View the details of a transaction
data.loc[0,'transactionItem']['instrument']

{'symbol': 'SPY_042222C370',
 'underlyingSymbol': 'SPY',
 'optionExpirationDate': '2022-04-22T05:00:00+0000',
 'putCall': 'CALL',
 'cusip': '0SPY..DM20370000',
 'description': 'SPY Apr 22 2022 370.0 Call',
 'assetType': 'OPTION'}

## Place an Order


Refer to the [Place Order APIs](https://developer.tdameritrade.com/account-access/apis) for detailed explanations and [examples](https://developer.tdameritrade.com/content/place-order-samples).

### Place Saved Orders

- Accounts with **advanced features** granted cannot place **saved orders** using api. 
- **Advanced features** for TDAmeritrade account refer to **futures** and **forex**. Need to contact customer support to turn off the features.

In [18]:
# define the endpoint
endpoint = f'https://api.tdameritrade.com/v1/accounts/{account_number}/savedorders'

# prepare the headers (per the docs)
# we need 'Content-Type' as `json` because it's required to pass payload (data) as json
headers = {'Authorization': f'Bearer {access_token}',
           'Content-Type': 'application/json'}

# order details
payload = {"orderType": "LIMIT",
        "price": "12.00",
        "session": "SEAMLESS", #pre, post and regular sessions
        "duration": "GOOD_TILL_CANCEL",
        "orderStrategyType": "SINGLE",
        "orderLegCollection": [{"instruction": "Buy",
                                "quantity": 1,
                                "instrument": {
                                  "symbol": "PLTR",
                                  "assetType": "EQUITY"
                                 }}]
}

# make a request (use `POST` method)
r = requests.post(url=endpoint, headers=headers, json=payload)
r

<Response [400]>

In [19]:
# see the error message
r.json()

{'error': 'Saved Orders are not supported for this account.'}

### Place Instant Orders

**Note:** Placing an instant order doesn't return any content data, unless there is error or rejection message. However, we can retrieve the order information using **get order** endpoint.

In [20]:
# define the endpoint
endpoint = f'https://api.tdameritrade.com/v1/accounts/{account_number}/orders'

# prepare the headers (per the docs)
# we need 'Content-Type' as `json` because it's required to pass payload (data) as json
headers = {'Authorization': f'Bearer {access_token}',
           'Content-Type': 'application/json'}

# order details
payload = {"orderType": "LIMIT",
        "price": "6.00",
        "session": "SEAMLESS", #pre, post and regular sessions
        "duration": "GOOD_TILL_CANCEL",
        "orderStrategyType": "SINGLE",
        "orderLegCollection": [{"instruction": "Buy",
                                "quantity": 1,
                                "instrument": {
                                  "symbol": "PLTR",
                                  "assetType": "EQUITY"
                                 }}]
}

# make a request (use `POST` method)
r = requests.post(url=endpoint, headers=headers, json=payload)
r

<Response [201]>

In [21]:
r.content

b''

## Get Orders

- Somehow I can't use [get orders by query](https://developer.tdameritrade.com/account-access/apis/get/orders-0) endpoint to retrieve orders, the error says "you don't have permission to access the resources")

- However, I can use [get orders by path](https://developer.tdameritrade.com/account-access/apis/get/accounts/%7BaccountId%7D/orders-0) to retrieve orders by an account.

In [22]:
# define the endpoint
endpoint = f'https://api.tdameritrade.com/v1/accounts/{account_number}/orders'

# prepare the headers (per the docs)
headers = {'Authorization': f'Bearer {access_token}'}

# query parameters
params = {'fromEnteredTime': '2022-02-17'}

# make a request (use `POST` method)
r = requests.get(url=endpoint, headers=headers, params=params)
r

<Response [200]>

In [23]:
# parse the data and convert to pandas df
orders = pd.DataFrame(r.json())

In [24]:
# check available fields
orders.columns

Index(['session', 'duration', 'orderType', 'complexOrderStrategyType',
       'quantity', 'filledQuantity', 'remainingQuantity',
       'requestedDestination', 'destinationLinkName', 'price',
       'orderLegCollection', 'orderStrategyType', 'orderId', 'cancelable',
       'editable', 'status', 'enteredTime', 'tag', 'accountId', 'closeTime',
       'orderActivityCollection'],
      dtype='object')

In [25]:
orders.sort_values('enteredTime', ascending=False, inplace=True)

In [26]:
# how orders information look like
cols = ['session', 'duration', 'orderType','quantity', 'filledQuantity', 'remainingQuantity',
       'price','orderId', 'cancelable','editable', 'status', 'enteredTime', 'tag', 'closeTime']
orders[cols].head()

,session,duration,orderType,quantity,filledQuantity,remainingQuantity,price,orderId,cancelable,editable,status,enteredTime,tag,closeTime
0,SEAMLESS,GOOD_TILL_CANCEL,LIMIT,1.0,0.0,1.0,6.00,5428534769,True,True,WORKING,2022-03-15T19:31:02+0000,API_OMS_REST:AA_catelinnx2,NaN
1,SEAMLESS,GOOD_TILL_CANCEL,LIMIT,1.0,0.0,0.0,6.00,5428432702,False,False,CANCELED,2022-03-15T19:15:11+0000,API_OMS_REST:AA_catelinnx2,2022-03-15T19:15:23+0000
2,SEAMLESS,GOOD_TILL_CANCEL,LIMIT,1.0,0.0,0.0,6.00,5428381769,False,False,CANCELED,2022-03-15T19:07:43+0000,API_OMS_REST:AA_catelinnx2,2022-03-15T19:07:57+0000
3,SEAMLESS,GOOD_TILL_CANCEL,LIMIT,1.0,0.0,0.0,6.00,5427839141,False,False,CANCELED,2022-03-15T17:10:20+0000,API_OMS_REST:AA_catelinnx2,2022-03-15T17:19:47+0000
4,NORMAL,DAY,NET_DEBIT,1.0,1.0,0.0,1.58,5427594584,False,False,FILLED,2022-03-15T16:22:18+0000,tIP,2022-03-15T16:22:21+0000


## Cancel Orders

In [27]:
# show all pending orders placed through api
def isPendingAPIorder(orders):
    return (orders['tag'].str.contains(r'API_OMS_REST*')) &\
           ((orders['status'] == 'WORKING') | \
            (orders['status'] == 'QUEUED'))

pending_api_orders = orders[isPendingAPIorder(orders)]
pending_api_orders.orderId

0    5428534769
Name: orderId, dtype: int64

In [28]:
# Now, let's cancel the above orders

# define endpoint
account_orders_url = f'https://api.tdameritrade.com/v1/accounts/{account_number}/orders/'

# define header
headers = {'Authorization': f'Bearer {access_token}'}

# cancel orders one by one
for orderId in pending_api_orders.orderId:
        r = requests.delete(account_orders_url+str(orderId), headers=headers)
        print(r.content) #print to see if any error message

b''


In [29]:
# let's see if those orders removed
# define the endpoint
endpoint = f'https://api.tdameritrade.com/v1/accounts/{account_number}/orders'

# prepare the headers (per the docs)
headers = {'Authorization': f'Bearer {access_token}'}

# query parameters
params = {'fromEnteredTime': '2022-02-17'}

# make a request (use `POST` method)
r = requests.get(url=endpoint, headers=headers, params=params)

# parse the data
orders = pd.DataFrame(r.json()).sort_values('enteredTime', ascending=False)

# check if any more pending orders placed through api
pending_api_orders = orders[isPendingAPIorder(orders)]
pending_api_orders.orderId

Series([], Name: orderId, dtype: int64)